In [16]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score

In [17]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [18]:
z = {'A': 0, 'B': 1, 'C': 2}
train_data.var2 = [z[item] for item in train_data.var2]
test_data.var2 = [z[item] for item in test_data.var2]

In [19]:
print(train_data['var2'].value_counts(), '\n')
print(test_data['var2'].value_counts())


0    25239
2     1040
1      217
Name: var2, dtype: int64 

0    8177
2     324
1      67
Name: var2, dtype: int64


In [20]:
train_data['datetime'] = pd.to_datetime(train_data['datetime'])
train_data.set_index('datetime', inplace=True)

test_data['datetime'] = pd.to_datetime(test_data['datetime'])
test_data.set_index('datetime', inplace=True)


In [21]:
X = train_data.drop(['ID','electricity_consumption'], axis = 1)
y = train_data['electricity_consumption']
test_data = test_data.drop('ID', axis = 1)

In [22]:
tscv = TimeSeriesSplit(n_splits = 100)

In [23]:
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [24]:
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.transform(X_test)

In [25]:
knn = KNeighborsRegressor()

In [26]:
knn.fit(X_scaled_train, y_train)
y_pred = knn.predict(X_scaled_test)

In [27]:
R2 = r2_score(y_test, y_pred)
print(f'R2 Score: {R2}')

R2 Score: 0.40052368156517204


In [28]:
test_pred = knn.predict(scaler.transform(test_data))

In [29]:
test_data['consumption_predictions'] = test_pred

In [30]:
test_data

,temperature,var1,pressure,windspeed,var2,consumption_predictions
datetime,,,,,,
2013-07-24 00:00:00,-10.0,-16.4,1011.0,263.280,0,205.8
2013-07-24 01:00:00,-10.0,-20.7,1011.0,267.175,0,204.0
2013-07-24 02:00:00,-10.7,-17.1,1003.0,269.555,0,201.6
2013-07-24 03:00:00,-13.6,-20.7,1008.0,273.060,0,199.8
2013-07-24 04:00:00,-10.7,-17.1,1006.0,1.765,0,243.6
...,...,...,...,...,...,...
2017-06-30 19:00:00,-5.7,-18.6,998.0,233.595,0,204.6
2017-06-30 20:00:00,-5.7,-17.1,995.0,238.780,0,202.2
2017-06-30 21:00:00,-7.1,-19.3,1004.0,244.325,0,205.2
